
# 4. Generation of metadata
create model
scripts to generate input files from VCF files

[ ] Make sure all <project_name = 'projectname'> is set to the projects name (in this case its soyadapt_data_analysis)
[ ] whats the name of my data?  set at data_name = collected_start_data for example.

Make sure dependencies and env are as it should be

ppp_env_x86.yaml / soyadapt_env_x86

In [1]:
import sys
import os
import subprocess

from pgpipe import four_gamete, vcf_split_pysam, vcf_to_ima, vcf_filter, vcf_calc, vcf_sampler, vcf_phase, stat_sampler, vcf_split, vcf_utilities, vcf_to_treemix, model_creator
from pgpipe.logging_module import initLogger
from pgpipe.informative_loci_filter import filter_bed_regions
from pgpipe.subtract_bed import filter_stat
import pysam

print ("Imports complete")

Imports complete


In [2]:
#define project name
#project_name = 'projectname'
project_name = 'soyadapt_data_analysis' # [ ] 

import os.path
path = '/Users/josephinelovemachine'
full_path = os.path.expanduser(path)
print (full_path)
# set home directory
# ~/<projectname>/
home_dir = os.path.join( full_path, project_name)
print (home_dir)
print ("home directory defined")


/Users/josephinelovemachine
/Users/josephinelovemachine/soyadapt_data_analysis
home directory defined


In [8]:
#Set up directories and filepaths, run on all restarts
config_dir = home_dir + '/config/'
results_dir = home_dir + '/results/'
workflow_dir = home_dir + '/workflow/' 
notebook_dir = workflow_dir + '/notebooks/' #WHE RE THIS NOTEBOOK SHOULD BE [ ]
# my working dir is where my data is ~/project_name/resources/
work_dir = home_dir + '/resources/'
#start data files:'
data_dir= work_dir +'start_data/' # my start data is here
#generated data files:
data_generated = work_dir + 'generated_data/'
#generated data files:
logfile = data_generated+'logfiles/'

#                 ****** this needs to change everywhere! ******
#Define start data name and location:
# data names are set to data _ name _ *** (e.g. data_name_snp)
# data locations are set to data _ *** _ datatype (e.g. data_snp_vcf)
#Which data do you want to use?
data_name_snp = 'soysnp50k' 
data_snp_vcf = data_dir+'soysnp50k.vcf.gz' 
data_name_test = 'test_data' 
data_test_vcf = data_dir+'test_data.vcf.gz' 
data_name_intersection = 'intersection_data'
data_intersection_vcf = data_dir+'intersection_data.vcf.gz' 
data_name_imputed = 'imputed_data'
data_imputed_vcf = data_dir+'imputed_data.vcf.gz' 

#Metadata

#chromosomes list
list_vcf_name_chr = data_dir+'soysnp50k_list_vcf_name_chr'
# string of all 20 chromosomes named chr
chr_all = data_dir+'chr.txt' 
# The 50k snps list 
list_snp_locations_50k = data_dir+'soysnp50k_ist_snp_locations_50k'
# metadata list:
list_vcf_name_country = data_dir+'soysnp50k_list_vcf_name_country'
list_vcf_name_maturitygroup = data_dir+'soysnp50k_list_vcf_name_maturitygroup'
#list of accessions + population(CCA/SA/Founders) + country + maturity group
list_vcf_name_collected = data_dir+'soysnp50k_list_vcf_name_collected'
#data files lists so they are named accordingly when generated or added to the project:
list_vcf_name_sa= data_dir+'soysnp50k_list_vcf_name_sa'
list_vcf_name_cca= data_dir+'soysnp50k_list_vcf_name_cca'
list_vcf_name_founders= data_dir+'soysnp50k_list_vcf_name_founders'

loci=50000
# print check directories and files
print(work_dir)
print(data_dir)
print(data_generated)


SyntaxError: invalid syntax (215423722.py, line 15)

In [5]:
#Set up directory structure, only needs to be run once
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
if not os.path.exists(workflow_dir):
    os.makedirs(workflow_dir)
if not os.path.exists(notebook_dir):
    os.makedirs(notebook_dir)            
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
if not os.path.exists(work_dir+'generated_data/'):
    os.makedirs(work_dir+'generated_data/')
if not os.path.exists(data_dir):
    os.makedirs(work_dir+'start_data/')
if not os.path.exists(data_generated):
    os.makedirs(data_generated)
if not os.path.exists(data_generated+'logfiles/'):
    os.makedirs(data_generated+'logfiles/')
# add files for the project that dont need and workflow access
if not os.path.exists(workflow_dir+'envs/'): # envs for each workflow
    os.makedirs(workflow_dir+'envs/')
if not os.path.exists(workflow_dir+'scripts/'): # scripts for the project like R scripts
    os.makedirs(workflow_dir+'scripts/')
if not os.path.exists(workflow_dir+'notebooks/'): # this notebook is here! move now! 
    os.makedirs(workflow_dir+'notebooks/')
if not os.path.exists(workflow_dir+'reports/'): # if not in logfile, then here
    os.makedirs(workflow_dir+'reports/')

move this notebook now to the notebook directory


a file with accessions, names, country, maturity 
Model files generation
chr.txt. 


file generation

make a file with the chromosomes.
there is also one called edit_chr.txt with old_names and new_names.

In [ ]:
for CHR in {1..20} ; do
    echo ${CHR} chr${CHR}
done >> chr_names.txt
#makes a list of 1-20 and 20 chromosomes named chr01 - chr20.

In [ ]:
for i in {01..20}; do
  echo "Chr$i Gm$i"
done > chr_names.txt

for CHR in {1..20} ; do
    echo ${CHR} chr${CHR}
done >> chr_names.txt
#makes a list of 1-20 and 20 chromosomes named chr01 - chr20.

In [ ]:
for i in {001..584}; do
  echo "${i}A"
done > accession_numbers_A.txt